# Analysis of links between behaviour and mechanical signals

In [1]:
import pandas as pd
import os.path
import numpy as np
import glob
from matplotlib import pyplot as plt
import seaborn as sns
import pingouin as pg
import openpyxl
import re
from scipy import signal

&copy; Fouad Baamal, JJ Aucouturier, 2025

## Meca data resample

In [2]:
for i in glob.glob("./data/meca/series/*") :
    print(i)

./data/meca/series\Sujet 06
./data/meca/series\Sujet 07
./data/meca/series\Sujet 08
./data/meca/series\Sujet 09
./data/meca/series\Sujet 10
./data/meca/series\Sujet 11
./data/meca/series\Sujet 12
./data/meca/series\Sujet 13
./data/meca/series\Sujet 14
./data/meca/series\Sujet 15
./data/meca/series\Sujet 16
./data/meca/series\Sujet 17
./data/meca/series\Sujet 18
./data/meca/series\Sujet 19
./data/meca/series\Sujet 21
./data/meca/series\Sujet 22
./data/meca/series\Sujet 23
./data/meca/series\Sujet 24
./data/meca/series\Sujet 25
./data/meca/series\Sujet 26
./data/meca/series\Sujet 27


In [3]:
def resample_all_cols(df, sr, new_sr, time_col='Time [s]'):
    """
    Resample selected data columns of a DataFrame from sr to new_sr.
    
    Parameters:
    - df: input DataFrame
    - sr: original sampling rate (Hz)
    - new_sr: target sampling rate (Hz)
    - time_col: name of the time column
    
    Returns:
    - resampled DataFrame
    """
    # List of columns to resample
    data_cols = ['Displacement [mm]', 'Fn [N]','Ft [N]', 'COF []', 'Acceleration [m/s²]']
    data_cols = [col for col in data_cols if col in df.columns]
    
    # Time array
    time = np.array(df[time_col])
    n_samples = int((len(time) / sr) * new_sr)

    # Resample each data column
    resampled_data = {}
    for col in data_cols:
        resampled_data[col], new_time = signal.resample(
            np.array(df[col]), num=n_samples, t=time
        )
    
    # Construct new DataFrame
    resampled_df = pd.DataFrame(resampled_data)
    resampled_df[time_col] = new_time
    
    return resampled_df

In [4]:
cols = ['Time [s]', 'Displacement [mm]', 'Fn [N]', 'Ft [N]', 'COF []', 'Acceleration [m/s²]']

In [83]:
data = pd.DataFrame()
i = 0
for subject_files in glob.glob("./data/meca/series/*") :    # load all subjects 
    for serie in glob.glob("%s/*"%subject_files) :# load a single file of a Subject
        data_serie = pd.read_excel(serie)
        data_serie[cols] = data_serie[cols].apply(pd.to_numeric)
        resampled_data = resample_all_cols(data_serie, sr=5000, new_sr=100)
        resampled_data['subj'] = int(subject_files[-2:])
        resampled_data['block'] = int(serie[40])
        resampled_data['trial'] = int(serie[42:44])
        resampled_data['stim_order'] = int(serie[53])

        resampled_data['normalised_time'] = np.linspace(1,100,len(resampled_data))
        #resampled_data.periode = resampled_data.periode + 1
        data = pd.concat([data, resampled_data], ignore_index=True)  # add data subject to the previous one
    print("subject file processed : %s"%subject_files)
    if i == 1 :
        break
    i = i+1

subject file processed : ./data/meca/series\Sujet 06
subject file processed : ./data/meca/series\Sujet 07


In [84]:
data.shape

(31685, 11)

In [85]:
data

,Displacement [mm],Fn [N],Ft [N],COF [],Acceleration [m/s²],Time [s],subj,block,trial,stim_order,normalised_time
0,33.007270,0.117109,0.425245,3.861166,-0.387779,2.446800,6,1,0,1,1.000000
1,19.045868,0.125383,0.361608,2.907422,-0.111234,2.456847,6,1,0,1,3.152174
2,23.241016,0.126181,0.365237,2.884752,-0.066935,2.466894,6,1,0,1,5.304348
3,22.723104,0.125211,0.345498,2.824736,-0.025590,2.476940,6,1,0,1,7.456522
4,25.184270,0.117907,0.344786,2.885973,0.048197,2.486987,6,1,0,1,9.608696
...,...,...,...,...,...,...,...,...,...,...,...
31680,96.578504,0.031024,0.358055,20.883060,-0.005923,7.886024,7,4,24,2,87.625000
31681,59.940385,0.036305,0.359145,9.311405,0.048157,7.896139,7,4,24,2,90.718750
31682,63.995651,0.028531,0.354491,15.211054,0.050631,7.906255,7,4,24,2,93.812500
31683,74.527464,0.030487,0.351970,14.807951,0.097538,7.916370,7,4,24,2,96.906250


## Load Behavioural Data

In [86]:
data_df = pd.read_csv('data/behaviour/all_data.csv')
data_df.stim_order = data_df.stim_order +1 
data_df

,subj,trial,block,practice,condition,sex,age,date,data_file,texture_id,stim_order,diameter,opening,spacing,response,rt
0,6,0,1,False,soft,f,23,2022-11-10 08:22:05.219727,results/221110_08.22_data_subj6_block1_trial0.csv,E030,1,260.0,140.0,400.0,False,10.857
1,6,0,1,False,soft,f,23,2022-11-10 08:22:05.219727,results/221110_08.22_data_subj6_block1_trial0.csv,E064,2,50.0,140.0,190.0,True,10.857
2,6,1,1,False,soft,f,23,2022-11-10 08:22:05.219727,results/221110_08.22_data_subj6_block1_trial1.csv,E061,1,80.0,170.0,250.0,True,11.883
3,6,1,1,False,soft,f,23,2022-11-10 08:22:05.219727,results/221110_08.22_data_subj6_block1_trial1.csv,E006,2,260.0,20.0,280.0,False,11.883
4,6,2,1,False,soft,f,23,2022-11-10 08:22:05.219727,results/221110_08.22_data_subj6_block1_trial2.csv,E048,1,170.0,170.0,340.0,False,10.245
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4395,27,22,4,False,rug,m,20,2022-11-25 17:18:05.980838,results/221125_17.18_data_subj27_block4_trial2...,E078,2,80.0,110.0,190.0,False,8.466
4396,27,23,4,False,rug,m,20,2022-11-25 17:18:05.980838,results/221125_17.18_data_subj27_block4_trial2...,E038,1,110.0,80.0,190.0,True,7.617
4397,27,23,4,False,rug,m,20,2022-11-25 17:18:05.980838,results/221125_17.18_data_subj27_block4_trial2...,E076,2,50.0,50.0,100.0,False,7.617
4398,27,24,4,False,rug,m,20,2022-11-25 17:18:05.980838,results/221125_17.18_data_subj27_block4_trial2...,E044,1,170.0,140.0,310.0,True,7.534


## Join les deux dataframes

In [87]:
beha_meca_data = pd.merge(data_df, data, on=['subj', 'trial','block','stim_order'])
beha_meca_data

,subj,trial,block,practice,condition,sex,age,date,data_file,texture_id,...,spacing,response,rt,Displacement [mm],Fn [N],Ft [N],COF [],Acceleration [m/s²],Time [s],normalised_time
0,6,0,1,False,soft,f,23,2022-11-10 08:22:05.219727,results/221110_08.22_data_subj6_block1_trial0.csv,E030,...,400.0,False,10.857,33.007270,0.117109,0.425245,3.861166,-0.387779,2.446800,1.000000
1,6,0,1,False,soft,f,23,2022-11-10 08:22:05.219727,results/221110_08.22_data_subj6_block1_trial0.csv,E030,...,400.0,False,10.857,19.045868,0.125383,0.361608,2.907422,-0.111234,2.456847,3.152174
2,6,0,1,False,soft,f,23,2022-11-10 08:22:05.219727,results/221110_08.22_data_subj6_block1_trial0.csv,E030,...,400.0,False,10.857,23.241016,0.126181,0.365237,2.884752,-0.066935,2.466894,5.304348
3,6,0,1,False,soft,f,23,2022-11-10 08:22:05.219727,results/221110_08.22_data_subj6_block1_trial0.csv,E030,...,400.0,False,10.857,22.723104,0.125211,0.345498,2.824736,-0.025590,2.476940,7.456522
4,6,0,1,False,soft,f,23,2022-11-10 08:22:05.219727,results/221110_08.22_data_subj6_block1_trial0.csv,E030,...,400.0,False,10.857,25.184270,0.117907,0.344786,2.885973,0.048197,2.486987,9.608696
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31680,7,24,4,False,soft,m,21,2022-11-10 10:41:45.411461,results/221110_10.41_data_subj7_block4_trial24...,E023,...,280.0,False,12.630,96.578504,0.031024,0.358055,20.883060,-0.005923,7.886024,87.625000
31681,7,24,4,False,soft,m,21,2022-11-10 10:41:45.411461,results/221110_10.41_data_subj7_block4_trial24...,E023,...,280.0,False,12.630,59.940385,0.036305,0.359145,9.311405,0.048157,7.896139,90.718750
31682,7,24,4,False,soft,m,21,2022-11-10 10:41:45.411461,results/221110_10.41_data_subj7_block4_trial24...,E023,...,280.0,False,12.630,63.995651,0.028531,0.354491,15.211054,0.050631,7.906255,93.812500
31683,7,24,4,False,soft,m,21,2022-11-10 10:41:45.411461,results/221110_10.41_data_subj7_block4_trial24...,E023,...,280.0,False,12.630,74.527464,0.030487,0.351970,14.807951,0.097538,7.916370,96.906250


In [88]:
beha_meca_data.rename(columns=lambda x: x[0].lower() + x.strip().lower().replace(' ', '_').title()[1:].lower(),inplace=True)  # snake_case
beha_meca_data.rename(columns=lambda x: re.sub(r'[^a-z0-9_]', '', x.strip().lower().replace(' ', '_')), inplace=True)

In [89]:
beha_meca_data

,subj,trial,block,practice,condition,sex,age,date,data_file,texture_id,...,spacing,response,rt,displacement_mm,fn_n,ft_n,cof_,acceleration_ms,time_s,normalised_time
0,6,0,1,False,soft,f,23,2022-11-10 08:22:05.219727,results/221110_08.22_data_subj6_block1_trial0.csv,E030,...,400.0,False,10.857,33.007270,0.117109,0.425245,3.861166,-0.387779,2.446800,1.000000
1,6,0,1,False,soft,f,23,2022-11-10 08:22:05.219727,results/221110_08.22_data_subj6_block1_trial0.csv,E030,...,400.0,False,10.857,19.045868,0.125383,0.361608,2.907422,-0.111234,2.456847,3.152174
2,6,0,1,False,soft,f,23,2022-11-10 08:22:05.219727,results/221110_08.22_data_subj6_block1_trial0.csv,E030,...,400.0,False,10.857,23.241016,0.126181,0.365237,2.884752,-0.066935,2.466894,5.304348
3,6,0,1,False,soft,f,23,2022-11-10 08:22:05.219727,results/221110_08.22_data_subj6_block1_trial0.csv,E030,...,400.0,False,10.857,22.723104,0.125211,0.345498,2.824736,-0.025590,2.476940,7.456522
4,6,0,1,False,soft,f,23,2022-11-10 08:22:05.219727,results/221110_08.22_data_subj6_block1_trial0.csv,E030,...,400.0,False,10.857,25.184270,0.117907,0.344786,2.885973,0.048197,2.486987,9.608696
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31680,7,24,4,False,soft,m,21,2022-11-10 10:41:45.411461,results/221110_10.41_data_subj7_block4_trial24...,E023,...,280.0,False,12.630,96.578504,0.031024,0.358055,20.883060,-0.005923,7.886024,87.625000
31681,7,24,4,False,soft,m,21,2022-11-10 10:41:45.411461,results/221110_10.41_data_subj7_block4_trial24...,E023,...,280.0,False,12.630,59.940385,0.036305,0.359145,9.311405,0.048157,7.896139,90.718750
31682,7,24,4,False,soft,m,21,2022-11-10 10:41:45.411461,results/221110_10.41_data_subj7_block4_trial24...,E023,...,280.0,False,12.630,63.995651,0.028531,0.354491,15.211054,0.050631,7.906255,93.812500
31683,7,24,4,False,soft,m,21,2022-11-10 10:41:45.411461,results/221110_10.41_data_subj7_block4_trial24...,E023,...,280.0,False,12.630,74.527464,0.030487,0.351970,14.807951,0.097538,7.916370,96.906250


## Kernels :

In [90]:
df_melt = pd.melt(beha_meca_data,
        id_vars=['subj','trial','block','sex','age','condition','response', 'stim_order', 'texture_id', 'time_s','normalised_time',],
        value_vars=['fn_n','ft_n','cof_','acceleration_ms','diameter','opening','spacing'],
        var_name='feature')

In [91]:
df_melt

,subj,trial,block,sex,age,condition,response,stim_order,texture_id,time_s,normalised_time,feature,value
0,6,0,1,f,23,soft,False,1,E030,2.446800,1.000000,fn_n,0.117109
1,6,0,1,f,23,soft,False,1,E030,2.456847,3.152174,fn_n,0.125383
2,6,0,1,f,23,soft,False,1,E030,2.466894,5.304348,fn_n,0.126181
3,6,0,1,f,23,soft,False,1,E030,2.476940,7.456522,fn_n,0.125211
4,6,0,1,f,23,soft,False,1,E030,2.486987,9.608696,fn_n,0.117907
...,...,...,...,...,...,...,...,...,...,...,...,...,...
221790,7,24,4,m,21,soft,False,2,E023,7.886024,87.625000,spacing,280.000000
221791,7,24,4,m,21,soft,False,2,E023,7.896139,90.718750,spacing,280.000000
221792,7,24,4,m,21,soft,False,2,E023,7.906255,93.812500,spacing,280.000000
221793,7,24,4,m,21,soft,False,2,E023,7.916370,96.906250,spacing,280.000000


In [92]:
df_melt.sort_values(by=['subj','block','trial'])
df_melt

,subj,trial,block,sex,age,condition,response,stim_order,texture_id,time_s,normalised_time,feature,value
0,6,0,1,f,23,soft,False,1,E030,2.446800,1.000000,fn_n,0.117109
1,6,0,1,f,23,soft,False,1,E030,2.456847,3.152174,fn_n,0.125383
2,6,0,1,f,23,soft,False,1,E030,2.466894,5.304348,fn_n,0.126181
3,6,0,1,f,23,soft,False,1,E030,2.476940,7.456522,fn_n,0.125211
4,6,0,1,f,23,soft,False,1,E030,2.486987,9.608696,fn_n,0.117907
...,...,...,...,...,...,...,...,...,...,...,...,...,...
221790,7,24,4,m,21,soft,False,2,E023,7.886024,87.625000,spacing,280.000000
221791,7,24,4,m,21,soft,False,2,E023,7.896139,90.718750,spacing,280.000000
221792,7,24,4,m,21,soft,False,2,E023,7.906255,93.812500,spacing,280.000000
221793,7,24,4,m,21,soft,False,2,E023,7.916370,96.906250,spacing,280.000000
